<a href="https://colab.research.google.com/github/Natthaphatz007/bsc_dpdm24/blob/main/midterm_dpdm24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## การเตรียมข้อมูลประกอบด้วย กำจัด outlier, แก้ missing, normalization, one-hot-encoder และ อื่นๆ ที่นักศึกษาเห็นว่าเหมาะสม

# 1 เตรียมข้อมูลเพื่อทำนายราคาบ้าน (15 คะแนน)
เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone'``` จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school')``` จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ  7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# 2 จากตารางในข้อ 1 ตอบคำถามต่อไปนี้ (5 คะแนน)
1. zone ใดมีราคาซื้อขายบ้าน 'sell price' เฉลี่ยสูงที่สุด (1 คะแนน)
2. บ้านที่มีจำนวน 7-11 อยู่ในรัศมี 1 กิโลเมตรมากที่สุด ราคาเท่าไหร่ (2 คะแนน)
3. ตำบล (sub_district) ไหน ได้คะแนนทรัพย์ 'score' เฉลี่ยจากนักประเมินมากที่สุด (2 คะแนน)

In [ ]:
import pandas as pd
import numpy as np
import json
from geopy.distance import geodesic

In [ ]:
!pip install gspread

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 60.1 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0


In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [218]:
# Load house data using gspread
# Replace with the actual Google Sheet URL
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1QO6OiGqyr8i78MVnjyLBBtr5Y8AoBM1tHZRg0Ud5CW4/edit#gid=397845144'
# Open the Google Sheet file
sheet = gc.open_by_url(google_sheet_url).sheet1  # Access the first sheet (sheet1)
# Convert the sheet data to a Pandas DataFrame
house_df = pd.DataFrame(sheet.get_all_records())

In [219]:
house_df.head()

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,furniture,building_number,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,ไม่มี,,,,,,,,,
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,ไม่มี,,,,,,,,,
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,,,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,ไม่ระบุ,,,,,,,,,


เตรียมข้อมูลเพื่อทำนายราคาบ้านมือสอง ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก google drive โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 	'sub_district', 'zone' จากไฟล์ "Data House" - บ้านมือ 2 (5 คะแนน)

In [271]:
# prompt: ข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้
# 'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price', 'district', 'sub_district', 'zone'

# Select only the required columns
selected_columns = ['id','renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story',
                    'number_of_bedroom','number_of_bathroom','direction','furniture','usable_area',
                    'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone']
house_df_sel = house_df[selected_columns]

# Display the first few rows of the processed DataFrame
house_df_sel.head()

,id,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone
0,1,no,0,1,45.0,2,2,2,ไม่ระบุ,ไม่มี,244,2,390000,ชนบท,ศรีบุญเรือง,
1,4,no,0,0,88.0,1,1,1,ไม่ระบุ,ไม่มี,164,3,820000,น้ำพอง,ทรายมูล,
2,16,no,1,0,8.0,1,11,11,ไม่ระบุ,ไม่ระบุ,489,9,3260000,มัญจาคีรี,กุดเค้า,
3,18,no,0,0,61.6,2,3,3,ไม่ระบุ,ไม่ระบุ,203,2,5000000,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน
4,19,no,0,0,72.3,2,3,2,ไม่ระบุ,ไม่ระบุ,164,2,3500000,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน


In [232]:
house_df[['latitude', 'longitude']]

,latitude,longitude
0,16.139321,102.631587
1,16.676156,102.917998
2,16.129358,102.53067
3,16.459052,102.806404
4,16.45,102.80068
...,...,...
1651,16.48011,102.574033
1652,16.528551,102.411762
1653,15.867444,102.482085
1654,16.636912,102.105343


In [233]:
seven_eleven_df

,latitude,longitude
0,16.060315,102.726486
1,16.056474,102.726844
2,16.059541,102.729431
3,16.075200,102.735602
4,16.070648,102.734405
...,...,...
25049,13.740390,100.625289
25050,13.742310,100.624788
25051,13.760205,100.622040
25052,13.736292,100.616910


จำนวน 7-11 และ โรงเรียน ในรัศมี 1 กิโลเมตร (ตั้งชื่อ 'd1000-7/11' และ 'd1000-school') จากไฟล์ "7-eleven.json" และ "school.json" โดยคำนวณจากพิกัด latitute longitude ระหว่างบ้าน และ 7-11 หรือ โรงเรียน ใกล้เคียงของบ้านแต่ละ id ในไฟล์ json (id บ้านหลังไหนที่มีมีข้อมูลแปลว่าไม่มี 7-11 หรือ โรงเรียนอยู่ในรัศมี 5 กิโลเมตร) (7 คะแนน)

In [272]:
import pandas as pd
import json
import geopy.distance
import multiprocessing

# Load the house data
# Load the 7-eleven and school data from JSON files
with open("/content/drive/MyDrive/Data prepare & Data Mining/MIDterm/7-eleven.json", "r") as f:
    seven_eleven_data = json.load(f)

with open("/content/drive/MyDrive/Data prepare & Data Mining/MIDterm/school.json", "r") as f:
    school_data = json.load(f)

# Function to calculate the distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    return geopy.distance.distance((lat1, lon1), (lat2, lon2)).km

# Function to process each house's data
def process_house(house, seven_eleven_data, school_data):
    house_lat = house['latitude']  # Access latitude from the house Series
    house_lon = house['longitude']  # Access longitude from the house Series

    # Get the list of 7-Elevens for the current house ID
    stores = seven_eleven_data.get(str(house['id']), [])

    # Count the number of 7-11s within 1 km of the house
    seven_eleven_count = sum(
        1 for store in stores
        if 'latitude' in store and 'longitude' in store and haversine(house_lat, house_lon, store['latitude'], store['longitude']) <= 1 # Check if 'latitude' and 'longitude' keys exist
    )

    # Get the list of schools for the current house ID
    schools = school_data.get(str(house['id']), [])

    # Count the number of schools within 1 km of the house
    school_count = sum(
        1 for school in schools
        if 'latitude' in school and 'longitude' in school and haversine(house_lat, house_lon, school['latitude'], school['longitude']) <= 1 # Check if 'latitude' and 'longitude' keys exist
    )

    return (seven_eleven_count, school_count)

# Using multiprocessing to speed up the process
def compute_distances(house_df, seven_eleven_data, school_data):
    # Create a pool of workers for parallel processing
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.starmap(process_house, [(house, seven_eleven_data, school_data) for _, house in house_df.iterrows()])

    # Split the results into separate lists for 7-11 and school counts
    seven_eleven_counts, school_counts = zip(*results)

    # Add the counts as new columns in the DataFrame
    house_df_sel['d1000-7/11'] = seven_eleven_counts  # Add columns to house_df
    house_df_sel['d1000-school'] = school_counts  # Add columns to house_df

    return house_df_sel  # Return the updated house_df

# Calculate the distances and update the house_df
house_df_sel = compute_distances(house_df, seven_eleven_data, school_data)  # Update house_df

# Display the updated house_df
house_df_sel

<ipython-input-272-de0d86f739b3>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_df_sel['d1000-7/11'] = seven_eleven_counts  # Add columns to house_df
<ipython-input-272-de0d86f739b3>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  house_df_sel['d1000-school'] = school_counts  # Add columns to house_df


,id,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price,district,sub_district,zone,d1000-7/11,d1000-school
0,1,no,0,1,45.0,2,2,2,ไม่ระบุ,ไม่มี,244,2,390000,ชนบท,ศรีบุญเรือง,,0,0
1,4,no,0,0,88.0,1,1,1,ไม่ระบุ,ไม่มี,164,3,820000,น้ำพอง,ทรายมูล,,0,0
2,16,no,1,0,8.0,1,11,11,ไม่ระบุ,ไม่ระบุ,489,9,3260000,มัญจาคีรี,กุดเค้า,,0,0
3,18,no,0,0,61.6,2,3,3,ไม่ระบุ,ไม่ระบุ,203,2,5000000,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน,0,0
4,19,no,0,0,72.3,2,3,2,ไม่ระบุ,ไม่ระบุ,164,2,3500000,เมืองขอนแก่น,บ้านเป็ด,มะลิวัลย์-ร.8-สนามบิน,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,no,0,0,73.0,1,2,1,ไม่ระบุ,ไม่ระบุ,108.76,,437400,บ้านฝาง,หนองบัว,,0,0
1652,20462P,no,0,0,84.0,1,2,1,ไม่ระบุ,ไม่ระบุ,119,,714600,หนองเรือ,โนนทอง,,0,0
1653,20463P,no,0,1,45.0,2,1,1,ไม่ระบุ,ไม่ระบุ,188.5,,451000,แวงน้อย,ก้านเหลือง,,0,0
1654,20466P,no,0,1,59.0,2,1,1,ไม่ระบุ,ไม่ระบุ,207,,910000,ชุมแพ,วังหินลาด,,0,0


In [161]:
import pandas as pd

# สมมติ school_df คือ DataFrame ที่มีคอลัมน์ 'geometry'
# แยก lat และ lng จาก geometry
school_df['lat'] = school_df['geometry'].apply(lambda x: x['location']['lat'])
school_df['lng'] = school_df['geometry'].apply(lambda x: x['location']['lng'])
school_df = school_df[['lat', 'lng']]
school_df

,lat,lng
0,16.131187,102.650437
1,16.135064,102.629047
2,16.148207,102.620759
3,16.127920,102.613829
4,16.674452,102.918958
...,...,...
36605,13.749322,100.650787
36606,13.733208,100.644963
36607,13.764595,100.636494
36608,13.733252,100.640597


In [190]:
school_df['longitude'] = school_df['longtitude']
school_df = school_df[['latitude', 'longitude']]
school_df

,latitude,longitude
0,16.131187,102.650437
1,16.135064,102.629047
2,16.148207,102.620759
3,16.127920,102.613829
4,16.674452,102.918958
...,...,...
36605,13.749322,100.650787
36606,13.733208,100.644963
36607,13.764595,100.636494
36608,13.733252,100.640597


In [160]:
# สมมติ seven_eleven_df คือ DataFrame ที่มีคอลัมน์ 'geometry'
# แยก lat และ lng จาก geometry
seven_eleven_df['lat'] = seven_eleven_df['geometry'].apply(lambda x: x['location']['lat'])
seven_eleven_df['lng'] = seven_eleven_df['geometry'].apply(lambda x: x['location']['lng'])
seven_eleven_df = seven_eleven_df[['lat', 'lng']]
seven_eleven_df

,lat,lng
0,16.060315,102.726486
1,16.056474,102.726844
2,16.059541,102.729431
3,16.075200,102.735602
4,16.070648,102.734405
...,...,...
25049,13.740390,100.625289
25050,13.742310,100.624788
25051,13.760205,100.622040
25052,13.736292,100.616910


In [191]:
seven_eleven_df['latitude'] = seven_eleven_df['lat']
seven_eleven_df['longitude'] = seven_eleven_df['longtitude']
seven_eleven_df = seven_eleven_df[['latitude', 'longitude']]
seven_eleven_df

<ipython-input-191-b71d35671170>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seven_eleven_df['longitude'] = seven_eleven_df['longtitude']


,latitude,longitude
0,16.060315,102.726486
1,16.056474,102.726844
2,16.059541,102.729431
3,16.075200,102.735602
4,16.070648,102.734405
...,...,...
25049,13.740390,100.625289
25050,13.742310,100.624788
25051,13.760205,100.622040
25052,13.736292,100.616910


ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

In [262]:
# Load house data using gspread
# Replace with the actual Google Sheet URL
result_id_url = 'https://docs.google.com/spreadsheets/d/1A3A-Yl4Tmcq2kPWf37QG6xdRZZ-4XDWULRPfORGPrNQ/edit?gid=905447057#gid=905447057'
# Open the Google Sheet file
sheet1 = gc.open_by_url(result_id_url).sheet1  # Access the first sheet (sheet1)
# Convert the sheet data to a Pandas DataFrame
appraiser_df = pd.DataFrame(sheet1.get_all_records())

In [263]:
appraiser_df.head()

,ID ผู้ประเมิน,post id,ID,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,ประเมินราคาซื้อ-ขาย,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,ให้คะแนนทรัพย์
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000,2500000,"5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000,2700000,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000,2200000,"5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000,2400000,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000,5000000,"5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [264]:
appraiser_df.rename(columns={'ID ผู้ประเมิน': 'Appraiser ID'}, inplace=True)

In [265]:
appraiser_df.rename(columns={'ประเมินราคาซื้อ-ขาย': 'estimator'}, inplace=True)
appraiser_df.rename(columns={'ให้คะแนนทรัพย์': 'score'}, inplace=True)

In [266]:
appraiser_df.rename(columns={'ID': 'id'}, inplace=True)

In [267]:
appraiser_df.head()

,Appraiser ID,post id,id,property_type,ความมั่นใจในการเมินราคาทรัพย์,ราคาทรัพย์,estimator,ปัจจัยในการประเมินราคาซื้อ-ขาย,เหตุผลไม่มั่นใจราคาประเมิน,score
0,1,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000,2500000,"5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
1,2,https://nayoo.co/khonkaen/posts/25,25,บ้านเดี่ยว,มั่นใจ,2890000,2700000,"โครงการนี้ราคาเสนอขาย 2,500,000-3,000,000 บาท/...",,4 ขายได้ ใช้ระยะเวลา 4 - 6 เดือน
2,1,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000,2200000,"5,000-15,000",,2 ขายได้ยาก ใช้ระยะเวลามากว่า 1 ปี
3,2,https://nayoo.co/khonkaen/posts/33,33,บ้านเดี่ยว,มั่นใจ,2600000,2400000,"ราคาเสนอขายประมาณ 2,200,000 - 2,500,000 บาท/หลัง",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน
4,1,https://nayoo.co/khonkaen/posts/495,495,บ้านเดี่ยว,มั่นใจ,6050000,5000000,"5,000-15,000",,3 ขายได้ ใช้ระยะเวลา 7 - 12 เดือน


In [273]:
# prompt: ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score') จากไฟล์ "คำตอบนักประเมิน(ID)" - บ้าน โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว (3 คะแนน)

# Filter for appraiser ID 01
appraiser_01_df = appraiser_df[appraiser_df['Appraiser ID'] == '01']

# Merge with house_df instead of house_df_sel as 'id' column is likely in house_df
house_df_sel2 = pd.merge(house_df_sel, appraiser_01_df[['id', 'estimator', 'score']], on='id', how='left')

# Now select the desired columns from the merged DataFrame
selected_columns2 = ['id','renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story',
                    'number_of_bedroom','number_of_bathroom','direction','furniture','usable_area',
                    'number_of_parking', 'sell_price', 'district', 'sub_district', 'zone','estimator','score', 'd1000-7/11', 'd1000-school']
house_df_sel2 = house_df_sel2[selected_columns2]

In [274]:
import pandas as pd

# สมมติว่าได้ทำการแก้ไขปัญหาข้อมูลเรียบร้อยแล้ว

# Merge with house_df_sel
house_df_sel2 = pd.merge(house_df, appraiser_01_df[['id', 'estimator', 'score']], on='id', how='left')

# ตรวจสอบผลลัพธ์
print(house_df_sel2.head())

# หากยังมีค่า NaN อยู่ อาจต้องทำการเติมค่า (imputation) หรือลบแถวที่มีค่า NaN ออก
# เลือกวิธีที่เหมาะสมตามลักษณะของข้อมูลและวัตถุประสงค์ของการวิเคราะห์

# ตัวอย่างการเติมค่าด้วยค่าเฉลี่ย (หากเหมาะสม)
house_df_sel2['estimator'].fillna(house_df_sel2['estimator'].mean(), inplace=True)
house_df_sel2['score'].fillna(house_df_sel2['score'].mean(), inplace=True)

   id                                   name_th  \
0   1                   บ้านเดี่ยว ชนบท ขอนแก่น   
1   4                 บ้านเดี่ยว น้ำพอง ขอนแก่น   
2  16              บ้านเดี่ยว มัญจาคีรี ขอนแก่น   
3  18     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   
4  19  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม   

                                    name_en    project_name  \
0                   บ้านเดี่ยว ชนบท ขอนแก่น     บ้านวังเวิน   
1                 บ้านเดี่ยว น้ำพอง ขอนแก่น  บ้านหนองบัวบาน   
2              บ้านเดี่ยว มัญจาคีรี ขอนแก่น                   
3     บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา   ชลลดา ขอนแก่น   
4  บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม        เบสท์โฮม   

                                 company province      district sub_district  \
0                                         ขอนแก่น          ชนบท  ศรีบุญเรือง   
1                                         ขอนแก่น        น้ำพอง      ทรายมูล   
2                                         ขอนแก่น     มัญจาคีรี      กุดเค

<ipython-input-274-dd4bf72a5449>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  house_df_sel2['estimator'].fillna(house_df_sel2['estimator'].mean(), inplace=True)
<ipython-input-274-dd4bf72a5449>:15: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  house_df_sel2['estimator'].fillna(house_df_sel2['estimator'].me

In [275]:
house_df_sel2.columns

Index(['id', 'name_th', 'name_en', 'project_name', 'company', 'province',
       'district', 'sub_district', 'property_type', 'home improvement',
       'renovate', 'public utility', 'land type', 'land wide(m)',
       'land long(m)', 'topsoil', 'corner', 'zip_code', 'zone', 'alley',
       'street', 'latitude', 'longitude', 'number_of_rai', 'number_of_ngan',
       'number_of_va', 'construct_year', 'model', 'sell_price',
       'number_of_unit', 'number_of_story', 'number_of_bedroom',
       'number_of_bathroom', 'number_of_parking', 'usable_area',
       'central_cost', 'direction', 'furniture', 'building_number',
       'number_of_building', 'floor_number', 'parking_percentage', 'has_pool',
       'has_keycard', 'has_security', 'has_fitness', 'has_garden', 'estimator',
       'score'],
      dtype='object')

In [277]:
house_df_sel2

,id,name_th,name_en,project_name,company,province,district,sub_district,property_type,home improvement,...,number_of_building,floor_number,parking_percentage,has_pool,has_keycard,has_security,has_fitness,has_garden,estimator,score
0,1,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านเดี่ยว ชนบท ขอนแก่น,บ้านวังเวิน,,ขอนแก่น,ชนบท,ศรีบุญเรือง,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
1,4,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านเดี่ยว น้ำพอง ขอนแก่น,บ้านหนองบัวบาน,,ขอนแก่น,น้ำพอง,ทรายมูล,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
2,16,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,บ้านเดี่ยว มัญจาคีรี ขอนแก่น,,,ขอนแก่น,มัญจาคีรี,กุดเค้า,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
3,18,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านชลลดา,ชลลดา ขอนแก่น,บริษัท แลนด์แอนด์เฮ้าส์ จำกัด (มหาชน),ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
4,19,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,บ้านเดี่ยว เมืองขอนแก่น หมู่บ้านเบสต์โฮม,เบสท์โฮม,เบสท์โฮม เลี่ยงเมืองขอนแก่น,ขอนแก่น,เมืองขอนแก่น,บ้านเป็ด,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06026,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06027,,,ขอนแก่น,บ้านฝาง,หนองบัว,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
1652,20462P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06033,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-06034,,,ขอนแก่น,หนองเรือ,โนนทอง,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
1653,20463P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07581,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-07582,,,ขอนแก่น,แวงน้อย,ก้านเหลือง,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN
1654,20466P,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10402,ขายถูก! บ้านเดี่ยว จังหวัดขอนแก่น 02-88-10403,,,ขอนแก่น,ชุมแพ,วังหินลาด,บ้านเดี่ยว,no,...,,,,,,,,,NaN,NaN


In [279]:
# Convert 'sell_price' column to numeric, handling errors
house_df_sel2['sell_price'] = pd.to_numeric(house_df_sel2['sell_price'], errors='coerce')

# Replace NaN values with 0 (or another suitable value) if necessary
house_df_sel2['sell_price'] = house_df_sel2['sell_price'].fillna(0)

# Calculate average selling price by zone
avg_sell_price_by_zone = house_df_sel2.groupby('zone')['sell_price'].mean()

# ... (rest of your code)

In [291]:
house_df_sel2['sell_price'].max()

104884000.0

In [287]:
avg_sell_price_by_zone

,sell_price
zone,
,2.778959e+06
กลางเมือง-เมืองเก่า-กุดกว้าง,4.457461e+06
ดอนบม-บายพาส-บ้านผือ-พระลับ,3.293429e+06
ทุ่งสร้าง-ศรีจันทร์-ราชมงคล,5.351464e+06
ท่าพระ-หนองบัวดีหมี,2.764172e+06
บึงเนียม,4.587833e+06
บ้านทุ่ม-บ้านม่วง,2.474188e+06
บ้านเป็ด-โคกฟันโปง,3.179203e+06
ม.ภาค-บ้านกอก-บึงหนองโคตร,4.605049e+06
